<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/evolve_agi_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
evolve_agi.py

A simple mutation‐only evolutionary strategy to optimize
a parameter vector for any fitness function.

Usage (in a terminal or Colab cell):

    python evolve_agi.py --dim 10 --generations 200 --mutation_rate 0.05 --plot

"""

import numpy as np
import argparse
import matplotlib.pyplot as plt


def fitness_function(params: np.ndarray) -> float:
    """
    Example fitness: negative squared L2 norm.
    Maximizing this drives `params` toward the zero vector.
    """
    return -np.sum(params ** 2)


def evolve(params: np.ndarray,
           fitness_fn,
           mutation_rate: float = 0.1,
           generations: int = 100,
           record_history: bool = False):
    """
    Hill‐climbing evolution: start from `params`, apply Gaussian
    mutations, and keep any mutation that improves fitness.

    Args:
        params         Initial vector (shape [D]).
        fitness_fn     Callable f(params) → float (higher is better).
        mutation_rate  Std dev for gaussian noise.
        generations    Number of mutation attempts.
        record_history Whether to store best‐fitness over time.

    Returns:
        best_params, history (if record_history) else best_params
    """
    best_params = params.copy()
    best_fitness = fitness_fn(best_params)
    history = [best_fitness] if record_history else None

    for gen in range(1, generations + 1):
        candidate = best_params + mutation_rate * np.random.randn(*best_params.shape)
        f = fitness_fn(candidate)

        # Accept if improved
        if f > best_fitness:
            best_params, best_fitness = candidate, f

        if record_history:
            history.append(best_fitness)

    if record_history:
        return best_params, history
    return best_params


def parse_args():
    parser = argparse.ArgumentParser(
        description="Evolve an AGI‐style parameter vector by mutation and selection."
    )
    parser.add_argument(
        "--dim", type=int, default=10,
        help="Dimensionality of the parameter vector."
    )
    parser.add_argument(
        "--generations", type=int, default=100,
        help="Number of mutation trials."
    )
    parser.add_argument(
        "--mutation_rate", type=float, default=0.1,
        help="Std dev of gaussian noise for mutation."
    )
    parser.add_argument(
        "--seed", type=int, default=None,
        help="Random seed for reproducibility."
    )
    parser.add_argument(
        "--plot", action="store_true",
        help="Plot best‐fitness history at the end."
    )

    # In Colab/Jupyter ignore unrecognized flags like `-f`
    args, _ = parser.parse_known_args()
    return args


def main():
    args = parse_args()
    if args.seed is not None:
        np.random.seed(args.seed)

    # Initialize a random "AGI parameter" vector
    initial_params = np.random.randn(args.dim)

    # Evolve with history if plotting requested
    if args.plot:
        best_params, history = evolve(
            initial_params,
            fitness_function,
            mutation_rate=args.mutation_rate,
            generations=args.generations,
            record_history=True
        )
    else:
        best_params = evolve(
            initial_params,
            fitness_function,
            mutation_rate=args.mutation_rate,
            generations=args.generations,
            record_history=False
        )

    # Report results
    init_fit = fitness_function(initial_params)
    best_fit = fitness_function(best_params)
    print("\n=== Evolution Complete ===")
    print(f"Initial fitness : {init_fit:.4f}")
    print(f"Best    fitness : {best_fit:.4f}")
    print(f"Initial params  : {initial_params}")
    print(f"Evolved params  : {best_params}\n")

    # Plot fitness curve
    if args.plot:
        plt.figure(figsize=(6, 4))
        plt.plot(history, label="Best Fitness")
        plt.title("Evolution of Best Fitness")
        plt.xlabel("Generation")
        plt.ylabel("Fitness")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()


if __name__ == "__main__":
    main()